# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [31]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2021-08-04	Task offloading in Edge and Cloud Computing: A survey on mathematical, artificial intelligence and control theory solutions	Computer Networks	Next generation communication networks are expected to accommodate a high number of new and resource-voracious applications that can be offered to a large range of end users. Even though end devices are becoming more powerful, the available local resources cannot cope with the requirements of these applications. This has created a new challenge called task offloading, where computation intensive tasks need to be offloaded to more resource powerful remote devices. Naturally, the Cloud Computing is a well-tested infrastructure that can facilitate the task offloading. However, Cloud Computing as a centralized and distant infrastructure creates significant communication delays that cannot satisfy the requirements of the emerging delay-sensitive applications. To this end, the concept 

## Import pandas

We are using the very handy pandas library for dataframes.

In [32]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [33]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2021-08-04,Task offloading in Edge and Cloud Computing: A...,Computer Networks,Next generation communication networks are exp...,"Saeik, F., Avgeris, M., Spatharakis, D., Santi...",task2021,https://hal.science/hal-03243071/document
1,2018-06-18,Edge computing in IoT ecosystems for UAV-enabl...,2018 IEEE international conference on smart co...,Unmanned Aerial Vehicles (UAV) facilitate the ...,"Kalatzis, N., Avgeris, M., Dechouniotis, D., P...",edge2018,https://www.researchgate.net/profile/Marios-Av...
2,2019-02-03,Where there is fire there is smoke: A scalable...,Sensors,A Cyber-Physical Social System (CPSS) tightly ...,"Avgeris, M., Spatharakis, D., Dechouniotis, D....",smoke2019,https://www.mdpi.com/1424-8220/19/3/639/pdf
3,2019-01-27,Semantic interoperability for iot platforms in...,Sensors,One of the main obstacles towards the promotio...,"Kalatzis, N., Routis, G., Marinellis, Y., Avge...",semantic2019,https://www.mdpi.com/1424-8220/19/3/528/htm
4,2019-04-03,Adaptive resource allocation for computation o...,ACM Transactions on Internet Technology (TOIT),Although mobile devices today have powerful ha...,"Avgeris, M., Dechouniotis, D., Athanasopoulos,...",toit2019,https://pure.qub.ac.uk/files/159624593/acm_lat...
5,2020-10-01,Meson: Optimized cross-slice communication for...,IEEE Communications Magazine,Network slicing is set out to address crucial ...,"Papathanail, G., Pentelas, A., Fotoglou, I., P...",meson2020,https://ruomo.lib.uom.gr/bitstream/7000/861/1/...
6,2020-11-02,A switching offloading mechanism for path plan...,2020 International Conferences on Internet of ...,Industry 4.0 applications rely on mobile robot...,"Spatharakis, D., Avgeris, M., Athanasopoulos, ...",switching2020,https://pure.qub.ac.uk/files/217751250/1570660...
7,2022-01-15,ENERDGE: Distributed energy-aware resource all...,Sensors,Mobile applications are progressively becoming...,"Avgeris, M., Spatharakis, D., Dechouniotis, D....",enerdge2022,https://www.mdpi.com/1424-8220/22/2/660
8,2019-08-26,Single vision-based self-localization for auto...,2019 7th International Conference on Future In...,"We present a single vision-based, self-localiz...","Avgeris, M., Spatharakis, D., Athanasopoulos, ...",single2019,https://pureadmin.qub.ac.uk/ws/files/175176852...
9,2017-09-14,Semantic Resource Management of Federated IoT ...,"Ad-hoc, Mobile, and Wireless Networks: 16th In...",Testbeds and experimental network facilities a...,"Avgeris, M., Kalatzis, N., Dechouniotis, D., R...",semantic2017,https://www.researchgate.net/profile/Dimitrios...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [34]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [35]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [36]:
!ls ../_publications/

2015-10-01-paper-title-number-3.md 2020-10-01-meson2020.md
2017-09-14-semantic2017.md         2020-11-02-switching2020.md
2018-06-18-edge2018.md             2021-08-04-task2021.md
2018-10-23-utility2018.md          2021-09-07-enabling2021.md
2019-01-27-semantic2019.md         2021-09-21-thesis2021.md
2019-02-03-smoke2019.md            2022-01-07-resource2022.md
2019-04-03-toit2019.md             2022-01-15-enerdge2022.md
2019-08-26-single2019.md           2022-09-05-compuational2022.md


In [37]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
